In [1]:
from llvm_ml.torch import BasicBlockDataset
banned_ids = []
dataset = BasicBlockDataset("./data/ryzen3600_v11.cbuf", masked=True, banned_ids=banned_ids)
print(f"Training with {len(dataset)} samples")

Training with 44082 samples


In [ ]:
from model.GraphBERT import ThroughputEstimator
from torch_geometric.loader import DataLoader
import torch
from lightning.pytorch.loggers import TensorBoardLogger
import lightning.pytorch as pl
from model import GraphBERT
from lightning.pytorch.callbacks import ModelSummary
from torch_geometric import compile

import importlib
importlib.reload(GraphBERT)

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

batch_size = 128
hidden_dim = 64
emb_size = 16

num_training = int(0.7 * len(dataset))
num_val = len(dataset) - num_training

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_training, num_val])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=6, drop_last=True)

model = ThroughputEstimator(dataset.num_opcodes, emb_size, batch_size, hidden_dim, learning_rate=0.00001)
# model = compile(model)

logger = TensorBoardLogger("runs", name="bert")
logger.log_graph(model)
trainer = pl.Trainer(max_epochs=40, logger=logger, callbacks=[ModelSummary(max_depth=-1)])
trainer.fit(model, train_loader, val_loader)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/alex/miniconda3/envs/cpu-uarch-prediction2/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

   | Name                               | Type               | Params
---------------------------------------------------------------------------
0  | embedding                          | MCEmbedding        | 336 K 
1  | embedding.embedding                | Embedding          | 336 K 
2  | embedding.pos_encoding             | PositionalEncoding | 0     
3  | embedding.norm                     | LayerNorm      

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]